In [1]:
import os

temp_localfolder = '/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/VASP_temporary/'

# List directories
files = os.listdir(temp_localfolder)

In [2]:
# filter jobs
jobs = list(filter(lambda x: x.startswith('MD_Test_PMASS'),files))
n_jobs = len(jobs)

In [3]:
jobs[0]
#f'{temp_localfolder}{jobs[1]}/vasprun.xml'

'MD_Test_PMASS_1250amu_3x5x6_460eV_bulk_4b'

In [4]:
import pandas as pd
import py4vasp as pv
import numpy as np
import re 

data = pd.DataFrame({
    'Energy [eV]': 1000*[0],
    'Step': np.linspace(1,1000, 1000)
})

datapoints = []

for job in jobs:
    calc = pv.Calculation.from_path(f'{temp_localfolder}{job}')
    entry = calc.energy[:].read( "ETOTAL, temperature")
    pmass = re.search('PMASS_(.*)amu', job)
    pmass = pmass.group(1)

    datapoints.append(
        pd.DataFrame({
            'Energy [eV]': entry['total energy   ETOTAL'],
            'Temperature [K]': entry['temperature    TEIN'],
            'Step': np.linspace(1,1000,1000),
            'PMASS [amu]': pmass
        })
    )

results = pd.concat(datapoints)

In [5]:
import plotly.express as px

fig = px.line(results, x='Step', y='Energy [eV]', color='PMASS [amu]')
fig.show()

In [6]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# graphing 1x2
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

for setting in results['PMASS [amu]'].unique():
    subset = results.query('`PMASS [amu]`==@setting')
    fig.append_trace(go.Scatter(
                x=subset['Step'],
                y=subset['Energy [eV]'],
                name=f'PMASS [amu]: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>Final energy: %{y}",
            ),
            row=1, col=1
            )
    fig.append_trace(go.Scatter(
                x=subset['Step'],
                y=subset['Temperature [K]'],
                name=f'PMASS [amu]: {setting}',
                hovertemplate=
                    "%{text}"+"<br>Number of k-points: %{x}"+"<br>CPU time: %{y}",
            ),
            row=2, col=1
            )
fig.update_layout(
        title_text = 'PMASS testing',
        )

fig.update_xaxes(title_text='Step', row=2, col=1)
fig.update_yaxes(title_text='Energy [eV]', row=1, col=1)
fig.update_yaxes(title_text='Temperature [K]', row=2, col=1)

In [7]:
# save plot as HTML file
fig.write_html('/Users/dominicwelti/Library/CloudStorage/Dropbox/Master_Thesis/PMASS_testing.html')

In [8]:
calc.structure[:].plot()

NGLWidget(max_frame=999)